In [2]:
# import requests

# data={
#     "shop_id":1326
# }

# headers={
#     "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
#     "Referer":"http://www.hilife.com.tw/store_map.aspx?shop_id=1326",
#     "Cookie":"ASP.NET_SessionId=w03g5cz5rend44krf20aiemp; _ga=GA1.3.725101572.1513082040; _gid=GA1.3.448264806.1513082040; _gat=1",
#     "Host":"www.hilife.com.tw",
#     "Origin":"http://www.hilife.com.tw",
#     "Content-Type":"application/x-www-form-urlencoded",
#     "X-Requested-With":"XMLHttpRequest",
#     "Accept-Language":"en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7",
#     "Accept":"application/json, text/javascript, */*; q=0.01",
#     "Accept-Encoding":"gzip, deflate",
#     "Connection":"keep-alive",
#     "Content-Length":"34"
# }

# res = requests.post('http://www.hilife.com.tw/getGoogleSpot.ashx', data=data, headers=headers)
# res

In [1]:
import requests 
import json
import pprint
area_dict = {
    "桃園市":['中壢區','平鎮區','龍潭區','楊梅區','新屋區','觀音區','桃園區','龜山區','八德區','大溪區','大園區','蘆竹區'],
    "新竹縣":["新埔鎮","竹北市","湖口鄉","新豐鄉","芎林鄉","竹東鎮","尖石鄉","寶山鄉"],
    "新竹市":["東區","北區","香山區"],
    "苗栗縣":["竹南鎮","頭份市","後龍鎮","苑裡鎮","苗栗市","公館鄉","大湖鄉","銅鑼鄉","泰安鄉","三義鄉"]
}

shops_list=[]
for city in area_dict.keys():
    for district in area_dict[city]:
        data = {
        'city_name':city,
        'town_name':district,
        'shop_id':''
        }        
        shops = requests.post('http://www.hilife.com.tw/getGoogleSpot.ashx', data=data).text #shops為request下來的文字，格式為str，但我們觀察到她其實是一個list的樣子
        shops_list.extend(json.loads(shops)) #這個shops的字串經json.loads的解析後，會是一個list。然後我們在這一次的迭代把這一個list extend到shops_list中





In [2]:
import re
_id = re.findall('\'名稱\': \'萊爾富 (.{4})',str(shops_list))
name = re.findall('\'名稱\': \'(.{1,15})\'',str(shops_list))
long = re.findall('\'經度\': (.{1,25})\,',str(shops_list))
lati = re.findall('\'緯度\': (.{1,25})\,',str(shops_list))
add = re.findall('<br/>地址：(.{1,30})<br/>',str(shops_list))
tel=[]
for i in range(len(shops_list)):
    t = re.findall('<br/>電話：(.{1,20})<br/>',str(shops_list[i]))
    if len(t)==0:
        tel.append('03-3366995')
    else:
        tel.extend(t) 
add=[]
for i in range(len(shops_list)):
    a = re.findall('<br/>地址：(.{1,30})<br/>',str(shops_list[i]))
    if len(a)==0:
        add.append('666桃竹苗桃竹苗')
    else:
        add.extend(a)



In [3]:
shopsdict=[]
for i in range(1,len(name)):
    shopsdict.append({
        '_id':'hi_'+_id[i],
        'name':name[i],
        'address':add[i],
        'city':add[i][3:6],
        'district': add[i][6:8] if add[i][7]=='區' else add[i][6:9] ,                  
        'longitude': long[i],
        'latitude': lati[i],
        'tel': tel[i],
        'wc':'N'        
    })

In [4]:
#到本block程式已結束，剩下的是debug的痕跡
shopsdict

[{'_id': 'hi_4073',
  'address': '320桃園市中壢區合定路11號',
  'city': '桃園市',
  'district': '中壢區',
  'latitude': '24.988203048706055',
  'longitude': '121.24229431152344',
  'name': '萊爾富 4073 中壢合定店',
  'tel': '03-2860821',
  'wc': 'N'},
 {'_id': 'hi_4198',
  'address': '320桃園市中壢區龍東路750號',
  'city': '桃園市',
  'district': '中壢區',
  'latitude': '24.937910079956055',
  'longitude': '121.26060485839844',
  'name': '萊爾富 4198 中壢陸科店',
  'tel': '03-2860504',
  'wc': 'N'},
 {'_id': 'hi_4201',
  'address': '320桃園市中壢區新生路二段324、326、328號',
  'city': '桃園市',
  'district': '中壢區',
  'latitude': '24.982362747192383',
  'longitude': '121.22286224365234',
  'name': '萊爾富 4201 中壢聖德店',
  'tel': '03-2860273',
  'wc': 'N'},
 {'_id': 'hi_4187',
  'address': '320桃園市中壢區龍岡路二段326號',
  'city': '桃園市',
  'district': '中壢區',
  'latitude': '24.94116973876953',
  'longitude': '121.230224609375',
  'name': '萊爾富 4187 中壢翔嘉店',
  'tel': '03-2860406',
  'wc': 'N'},
 {'_id': 'hi_3917',
  'address': '320桃園市中壢區內定二街998之3號',
  'city': '桃園市',
  '

In [5]:
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
db = client["cuisines"]
db.cvs.insert_many(shopsdict)

In [104]:
for i in range(len(shops_list)):
    a = re.findall('<br/>地址：(.{1,30})<br/>',str(shops_list[i]))
    if len(a)==0:
        print(shops_list[i])
    else:
        pass
    

{'緯度': 25.02991485595703, '經度': 121.3900375366211, '名稱': '萊爾富 3749 桃縣長院店', '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 3749 桃縣長院店<br/>地址：333桃園市龜山區文化一路261號長庚技術學院第一宿舍壹樓部份<br/>電話：03-3971390<br/><img src='images/icon/marked_tscc.gif' width='25' height='25' title='悠遊卡加值' />&nbsp;&nbsp;<img src='images/icon/marked_coffee.gif' width='25' height='25' title='咖啡機' />&nbsp;&nbsp;<img src='images/icon/marked_lifeEt.gif' width='25' height='25' title='Life-ET(購票)服務' />&nbsp;&nbsp;<img src='images/icon/marked_ecard.gif' width='25' height='25' title='一卡通' />&nbsp;&nbsp;"}
{'緯度': 25.034576416015625, '經度': 121.39095306396484, '名稱': '萊爾富 3808 桃縣長青店', '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 3808 桃縣長青店<br/>地址：333桃園市龜山區文化一路259號 (長庚大學 據德樓 B1部份)<br/>電話：03-2118358<br/><img src='images/icon/marked_tscc.gif' width='25' height='25' title='悠遊卡加值' />&nbsp;&nbsp;<img src='images/icon/marked_lifeEt.gif' width='25' height='25' title='Life-ET(購票)服務' />&nbsp;&nbs

In [135]:
shops_list

[{'名稱': '萊爾富 4330 中壢文定店',
  '經度': 121.24990844726562,
  '緯度': 24.988935470581055,
  '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 4330 中壢文定店<br/>地址：320桃園市中壢區吉林北路6號全部<br/>電話：03-2860405<br/>"},
 {'名稱': '萊爾富 4073 中壢合定店',
  '經度': 121.24229431152344,
  '緯度': 24.988203048706055,
  '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 4073 中壢合定店<br/>地址：320桃園市中壢區合定路11號<br/>電話：03-2860821<br/><img src='images/icon/marked_atm.gif' width='25' height='25' title='提款服務' />&nbsp;&nbsp;<img src='images/icon/marked_tscc.gif' width='25' height='25' title='悠遊卡加值' />&nbsp;&nbsp;<img src='images/icon/marked_coffee.gif' width='25' height='25' title='咖啡機' />&nbsp;&nbsp;<img src='images/icon/marked_lifeEt.gif' width='25' height='25' title='Life-ET(購票)服務' />&nbsp;&nbsp;<img src='images/icon/marked_ecard.gif' width='25' height='25' title='一卡通' />&nbsp;&nbsp;"},
 {'名稱': '萊爾富 4198 中壢陸科店',
  '經度': 121.26060485839844,
  '緯度': 24.937910079956055,
  '說明': "<img src='images/goo

In [67]:
# 
for i in range(len(shops_list)):
    a = re.findall('<img.+title=\'(提款服務)\'',str(shops_list[i]['說明']))
    if len(a) < 1:
        a.append('N')
    else:
        a[0] = 'Y'
    print(a)

['N']
['Y']
['N']
['Y']
['N']
['N']
['Y']
['N']
['Y']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['N']
['N']
['N']
['Y']
['Y']
['N']
['N']
['N']
['N']
['Y']
['N']
['Y']
['N']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['N']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['N']
['Y']
['N']
['N']
['N']
['N']
['Y']
['N']
['N']
['N']
['Y']
['Y']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N']
['Y']
['N']
['N']
['N']
['N']
['N']
['Y']
['N']
['N']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['Y']
['N'

In [41]:
shops_list

[{'名稱': '萊爾富 4330 中壢文定店',
  '經度': 121.24990844726562,
  '緯度': 24.988935470581055,
  '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 4330 中壢文定店<br/>地址：320桃園市中壢區吉林北路6號全部<br/>電話：03-2860405<br/>"},
 {'名稱': '萊爾富 4073 中壢合定店',
  '經度': 121.24229431152344,
  '緯度': 24.988203048706055,
  '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 4073 中壢合定店<br/>地址：320桃園市中壢區合定路11號<br/>電話：03-2860821<br/><img src='images/icon/marked_atm.gif' width='25' height='25' title='提款服務' />&nbsp;&nbsp;<img src='images/icon/marked_tscc.gif' width='25' height='25' title='悠遊卡加值' />&nbsp;&nbsp;<img src='images/icon/marked_coffee.gif' width='25' height='25' title='咖啡機' />&nbsp;&nbsp;<img src='images/icon/marked_lifeEt.gif' width='25' height='25' title='Life-ET(購票)服務' />&nbsp;&nbsp;<img src='images/icon/marked_ecard.gif' width='25' height='25' title='一卡通' />&nbsp;&nbsp;"},
 {'名稱': '萊爾富 4198 中壢陸科店',
  '經度': 121.26060485839844,
  '緯度': 24.937910079956055,
  '說明': "<img src='images/goo

In [14]:
a = json.loads(shops)
type(a)
a[0]

{'名稱': '萊爾富 2724 苗縣苗義店',
 '經度': 120.7616195678711,
 '緯度': 24.39849853515625,
 '說明': "<img src='images/google_logo.gif' width='25' height='21'/>萊爾富 | 2724 苗縣苗義店<br/>地址：367苗栗縣三義鄉水美路210號<br/>電話：037-878515<br/><img src='images/icon/marked_atm.gif' width='25' height='25' title='提款服務' />&nbsp;&nbsp;<img src='images/icon/marked_tscc.gif' width='25' height='25' title='悠遊卡加值' />&nbsp;&nbsp;<img src='images/icon/marked_coffee.gif' width='25' height='25' title='咖啡機' />&nbsp;&nbsp;<img src='images/icon/marked_lifeEt.gif' width='25' height='25' title='Life-ET(購票)服務' />&nbsp;&nbsp;"}

In [8]:
_id[300]

'4048'

In [9]:
long[300]

'121.00328826904297'

In [6]:
len(_id)==len(name)==len

True

In [63]:
len(lati)

385

In [59]:
len(name)

334

In [60]:
len(long)

385

In [ ]:
len(tel)

In [ ]:
len(add)

In [64]:
shops

'  {    "緯度": 24.39849853515625,    "經度": 120.76161956787109,    "名稱": "萊爾富 2724 苗縣苗義店",    "說明": "<img src=\'images/google_logo.gif\' width=\'25\' height=\'21\'/>萊爾富 | 2724 苗縣苗義店<br/>地址：367苗栗縣三義鄉水美路210號<br/>電話：037-878515<br/><img src=\'images/icon/marked_atm.gif\' width=\'25\' height=\'25\' title=\'提款服務\' />&nbsp;&nbsp;<img src=\'images/icon/marked_tscc.gif\' width=\'25\' height=\'25\' title=\'悠遊卡加值\' />&nbsp;&nbsp;<img src=\'images/icon/marked_coffee.gif\' width=\'25\' height=\'25\' title=\'咖啡機\' />&nbsp;&nbsp;<img src=\'images/icon/marked_lifeEt.gif\' width=\'25\' height=\'25\' title=\'Life-ET(購票)服務\' />&nbsp;&nbsp;"  },  {    "緯度": 24.390083312988281,    "經度": 120.75848388671875,    "名稱": "萊爾富 3871 苗縣車亭店",    "說明": "<img src=\'images/google_logo.gif\' width=\'25\' height=\'21\'/>萊爾富 | 3871 苗縣車亭店<br/>地址：367苗栗縣三義鄉西湖村下湖5-10號1F部份<br/>電話：037-879362<br/><img src=\'images/icon/marked_atm.gif\' width=\'25\' height=\'25\' title=\'提款服務\' />&nbsp;&nbsp;<img src=\'images/icon/marked_tscc.gif\'

In [1]:
import re
a_list = str(shops_list).split("    ")
for  a in a_list:
    
    print(a)
# a = re.findall('{(.+)}',str(shops_list))
# a

NameError: name 'shops_list' is not defined

In [79]:
a = ["apple"]
b = ["banana"]
a.extend(b)
a

['apple', 'banana']

In [80]:
a = ["apple"]
b = "banana"
a.append(b)
a

['apple', 'banana']

In [178]:
a = ["apple"]
b = "banana"
a.extend(b)
a

['apple', 'b', 'a', 'n', 'a', 'n', 'a']

In [82]:
a = "apple"
b = "banana"
a+b

'applebanana'

In [83]:
a = ["apple"]
b = ["banana"]
a+b

['apple', 'banana']

In [85]:
a = ["apple"]
b = ["banana"]
a.append(b)
a

['apple', ['banana']]

In [86]:
a = ["apple"]
b = "banana"
b.append(a)
b

AttributeError: 'str' object has no attribute 'append'